In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Lokasi folder dataset yang sudah diekstrak
dataset_dir = r'D:\kuliah\Semester 7\TA 1\dataset\Motif Ulos Toba\I BLG TB 8'  # Ganti dengan path yang sesuai

# Tentukan path folder output untuk gambar hasil augmentasi
output_dir = r'D:\kuliah\Semester 7\TA 1\dataset\Motif Ulos Toba\augmentasi_I BLG TB 8'

# Cek apakah folder output ada, jika belum ada buat folder tersebut
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Membuat objek ImageDataGenerator dengan augmentasi gambar (rotasi dan flipping)
datagen = ImageDataGenerator(
    rotation_range=40,  # Rotasi gambar antara 0 hingga 40 derajat
    horizontal_flip=True,  # Flip gambar secara horizontal
    fill_mode='nearest'  # Isi piksel yang hilang setelah transformasi
)

# Loop untuk memproses semua gambar dalam folder
for filename in os.listdir(dataset_dir):
    # Pastikan hanya file gambar dengan ekstensi tertentu yang diproses
    if filename.endswith(('.jpg', '.png')):  
        image_path = os.path.join(dataset_dir, filename)  # Menentukan path gambar
        if os.path.exists(image_path):
            # Menyiapkan gambar untuk augmentasi
            img = tf.keras.preprocessing.image.load_img(image_path)  # Muat gambar
            x = tf.keras.preprocessing.image.img_to_array(img)  # Konversi ke array
            x = x.reshape((1,) + x.shape)  # Ubah bentuk array menjadi format batch

            # Menghasilkan dan menyimpan gambar-gambar hasil augmentasi
            i = 0
            for batch in datagen.flow(x, save_to_dir=output_dir, save_prefix='ulos', save_format='jpeg'):
                i += 1
                if i > 30:  # Batasi jumlah gambar yang dihasilkan
                    break
        else:
            print(f"File {image_path} tidak ditemukan.")


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np

# Fungsi untuk meningkatkan resolusi gambar menggunakan OpenCV (super-resolution)
def increase_resolution(image_path, scale_factor=2):
    img = cv2.imread(image_path)  # Baca gambar
    width = int(img.shape[1] * scale_factor)
    height = int(img.shape[0] * scale_factor)

    # Menggunakan interpolasi bilinear untuk meningkatkan resolusi
    increased_img = cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)

    # Simpan gambar yang telah diperbesar
    cv2.imwrite(image_path.replace('.jpg', '_increased_resolution.jpg').replace('.png', '_increased_resolution.png'), increased_img)

    return increased_img

# Fungsi untuk menghaluskan gambar (sharpening)
def sharpen_image(image):
    # Membuat kernel sharpening
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    
    # Terapkan kernel pada gambar
    sharpened = cv2.filter2D(image, -1, kernel)
    return sharpened

# Lokasi folder dataset yang sudah diekstrak
# Gunakan raw string (r'...') atau forward slashes untuk menghindari escape-sequence (\a -> bell)
dataset_dir = r'D:\kuliah\Semester 7\TA 1\dataset\Motif Ulos Toba\I BM TB 3'  # Ganti dengan path yang sesuai
# Normalisasi path agar konsisten di semua OS
dataset_dir = os.path.normpath(dataset_dir)

# Tentukan path folder output untuk gambar hasil augmentasi
output_dir = r'D:\kuliah\Semester 7\TA 1\dataset\Motif Ulos Toba\augmentasi_I BM TB 3'

# Cek apakah folder output ada, jika belum ada buat folder tersebut
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Membuat objek ImageDataGenerator dengan augmentasi gambar (rotasi dan flipping)
datagen = ImageDataGenerator(
    rotation_range=40,  # Rotasi gambar antara 0 hingga 40 derajat
    horizontal_flip=True,  # Flip gambar secara horizontal
    fill_mode='nearest'  # Isi piksel yang hilang setelah transformasi
)

# Loop untuk memproses semua gambar dalam folder
for filename in os.listdir(dataset_dir):
    # Pastikan hanya file gambar dengan ekstensi tertentu yang diproses
    if filename.endswith(('.jpg', '.png')):  
        image_path = os.path.join(dataset_dir, filename)  # Menentukan path gambar
        if os.path.exists(image_path):
            # Meningkatkan resolusi gambar sebelum augmentasi
            img_increased_res = increase_resolution(image_path)

            # Menyiapkan gambar untuk augmentasi
            img = tf.keras.preprocessing.image.load_img(image_path)  # Muat gambar
            x = tf.keras.preprocessing.image.img_to_array(img)  # Konversi ke array
            x = x.reshape((1,) + x.shape)  # Ubah bentuk array menjadi format batch

            # Menghasilkan dan menyimpan gambar-gambar hasil augmentasi
            i = 0
            # Ambil batch dari generator (tanpa save_to_dir) lalu simpan dan lakukan sharpening langsung
            for batch in datagen.flow(x, batch_size=1):
                i += 1

                # Konversi batch ke uint8 dan pastikan rentang nilai valid
                aug = np.clip(batch[0], 0, 255).astype(np.uint8)

                # Datagen menghasilkan gambar dalam format RGB; OpenCV memakai BGR
                aug_bgr = cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)

                # Terapkan sharpening langsung pada array gambar
                sharpened_img = sharpen_image(aug_bgr)

                # Simpan gambar yang telah di-sharpen
                save_path = os.path.join(output_dir, f'ulos_{i}.jpeg')
                cv2.imwrite(save_path, sharpened_img)

                if i >= 20:  # Batasi jumlah gambar yang dihasilkan
                    break
        else:
            print(f"File {image_path} tidak ditemukan.")
